In [1]:
import pandas as pd
import sqlite3
import os # Pour lister les fichiers

In [2]:
data_dir = 'data/'
db_path = 'olist.db'

conn = sqlite3.connect(db_path)

for file in os.listdir(data_dir):
    if file.endswith('.csv'):
        table_name = file.replace('olist_', '').replace('_dataset', '').replace('.csv', '')
        file_path = os.path.join(data_dir, file)
        
        df = pd.read_csv(file_path)
        
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f'Table {table_name} chargée avec {len(df)} enregistrements.')


Table sellers chargée avec 3095 enregistrements.
Table product_category_name_translation chargée avec 71 enregistrements.
Table orders chargée avec 99441 enregistrements.
Table order_items chargée avec 112650 enregistrements.
Table customers chargée avec 99441 enregistrements.
Table geolocation chargée avec 1000163 enregistrements.
Table order_payments chargée avec 103886 enregistrements.
Table order_reviews chargée avec 99224 enregistrements.
Table products chargée avec 32951 enregistrements.


In [3]:
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") 
tables = cursor.fetchall()

print(tables)

conn.close()

[('sellers',), ('product_category_name_translation',), ('orders',), ('order_items',), ('customers',), ('geolocation',), ('order_payments',), ('order_reviews',), ('products',)]


In [4]:
conn = sqlite3.connect(db_path)

query_join_test = """
SELECT 
    t1.order_status,
    t2.customer_city
FROM orders AS t1
JOIN customers AS t2 ON t1.customer_id = t2.customer_id
LIMIT 10;"""

df_join_test = pd.read_sql_query(query_join_test, conn)
df_join_test


,order_status,customer_city
0,delivered,sao paulo
1,delivered,barreiras
2,delivered,vianopolis
3,delivered,sao goncalo do amarante
4,delivered,santo andre
5,delivered,congonhinhas
6,invoiced,santa rosa
7,delivered,nilopolis
8,delivered,faxinalzinho
9,delivered,sorocaba


On va construire notre table RFM (Recency, Frequency, Monetary). Pour chaque vrai client (customer_unique_id), je veux savoir :

- Le nombre total de commandes qu'il a passées (Frequency).
- La date de sa dernière commande (Recency)
- Combien ce client a-t-il dépensé au total (Monetary)

In [11]:
import datetime as dt

conn = sqlite3.connect(db_path)

query_rfm = """
    SELECT
        c.customer_unique_id,
        COUNT(DISTINCT o.order_id) AS frequency,
        MAX(o.order_purchase_timestamp) AS last_purchase_date,
        SUM(oi.price) AS monetary
    FROM
        customers AS c
    JOIN
        orders AS o ON c.customer_id = o.customer_id
    JOIN
        order_items AS oi ON o.order_id = oi.order_id
    GROUP BY
        c.customer_unique_id;

"""

df_rfm_raw = pd.read_sql_query(query_rfm, conn)

conn.close()

print(f"Table RFM brute chargée. Nombre de clients : {len(df_rfm_raw)}")
print(df_rfm_raw.info())

df_rfm_raw['last_purchase_date'] = pd.to_datetime(df_rfm_raw['last_purchase_date'])

latest_date = df_rfm_raw['last_purchase_date'].max()
today = latest_date + dt.timedelta(days=1)

df_rfm_raw['recency'] = (today - df_rfm_raw['last_purchase_date']).dt.days

print("\n--- Table RFM avec Récence (en jours) ---")
print(df_rfm_raw[['customer_unique_id', 'recency', 'frequency', 'monetary']].head())

Table RFM brute chargée. Nombre de clients : 95420
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95420 entries, 0 to 95419
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_unique_id  95420 non-null  object 
 1   frequency           95420 non-null  int64  
 2   last_purchase_date  95420 non-null  object 
 3   monetary            95420 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.9+ MB
None

--- Table RFM avec Récence (en jours) ---
                 customer_unique_id  recency  frequency  monetary
0  0000366f3b9a7992bf8c76cfdf3221e2      116          1    129.90
1  0000b849f77a49e4a4ce2b2a4ca5be3f      119          1     18.90
2  0000f46a3911fa3c0805444483337064      542          1     69.00
3  0000f6ccb0745a6a4b88665a16c9f078      326          1     25.99
4  0004aac84e0df4da2b147fca70cf8255      293          1    180.00


La prochaine étape est la segmentation, comment comparer un client "116, 1, 129" à un client "25, 3, 500" ? C'est impossible, il faut normaliser.

Seulement, quand on regarde la colonne frequency, on voit que c'est très souvent à 1. C'est normal. Dans la plupart des e-commerces, plus de 80% des clients n'achètent qu'une seule fois. Si on essaie de couper "frequency" en 4 groupes (quartiles), ça va échouer, car les 3 premiers "seuils" (25%, 50%, 75%) sont tous à "1".

Pour recency et monetary (qui sont bien répartis), on utilisera un outil automatique de Pandas : pd.qcut (quartile cut).

Pour frequency (qui est "asymétrique"), on créera une règle manuelle (ex: 1 achat = 1, 2 achats = 2, 3+ achats = 4).

In [12]:
df_rfm = df_rfm_raw.copy()

# NOTE : Les labels sont INVERSÉS [4, 3, 2, 1], un 'recency' FAIBLE (ex: 10 jours) est MEILLEUR (score 4).
df_rfm['R_Score'] = pd.qcut(df_rfm['recency'], 4, labels=[4, 3, 2, 1])

df_rfm['M_Score'] = pd.qcut(df_rfm['monetary'], 4, labels=[1, 2, 3, 4], duplicates='drop')

df_rfm['F_Score'] = df_rfm['frequency'].apply(lambda x: 1 if x == 1 else (2 if x == 2 else (3 if x == 3 else 4)))

df_rfm['RFM_Score_str'] = df_rfm['R_Score'].astype(str) + df_rfm['F_Score'].astype(str) + df_rfm['M_Score'].astype(str)


print("\n--- Table RFM avec Scores (1-4) ---")
print(df_rfm[['customer_unique_id', 'recency', 'R_Score', 'frequency', 'F_Score', 'monetary', 'M_Score', 'RFM_Score_str']].head())


--- Table RFM avec Scores (1-4) ---
                 customer_unique_id  recency R_Score  frequency  F_Score  \
0  0000366f3b9a7992bf8c76cfdf3221e2      116       4          1        1   
1  0000b849f77a49e4a4ce2b2a4ca5be3f      119       4          1        1   
2  0000f46a3911fa3c0805444483337064      542       1          1        1   
3  0000f6ccb0745a6a4b88665a16c9f078      326       2          1        1   
4  0004aac84e0df4da2b147fca70cf8255      293       2          1        1   

   monetary M_Score RFM_Score_str  
0    129.90       3           413  
1     18.90       1           411  
2     69.00       2           112  
3     25.99       1           211  
4    180.00       4           214  


Client 0 (RFM_Score_str = 413) :

R=4 (Très récent)

F=1 (1 seul achat)

M=3 (A dépensé pas mal)

Profil Business : C'est un "Nouveau Client à Forte Valeur". Il vient d'arriver, n'a acheté qu'une fois, mais a dépensé beaucoup.

Maintenant nous allons répondre à la question : Peut-on prédire si un client sera un 'gros dépensier' (M=4) en se basant uniquement sur sa Récence (R) et sa Fréquence (F) ?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

df_rfm['R_Score'] = df_rfm['R_Score'].astype(int)
df_rfm['F_Score'] = df_rfm['F_Score'].astype(int)
df_rfm['M_Score'] = df_rfm['M_Score'].astype(int)

Y = df_rfm['high_value'] = df_rfm['M_Score'].apply(lambda x: 1 if x == 4 else 0)
X = df_rfm[['recency', 'frequency']]

print("--- Cible (Y) ---")
print(Y.value_counts(normalize=True))

print("\n--- Features (X) ---")
print(X.head())

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"\nDonnées prêtes : {len(X_train)} clients pour l'entraînement, {len(X_test)} pour le test.")

--- Cible (Y) ---
M_Score
0    0.750199
1    0.249801
Name: proportion, dtype: float64

--- Features (X) ---
   recency  frequency
0      116          1
1      119          1
2      542          1
3      326          1
4      293          1

Données prêtes : 76336 clients pour l'entraînement, 19084 pour le test.


In [14]:
model = LogisticRegression(random_state=42)

print("Entraînement du modèle...")
model.fit(X_train, Y_train)
print("Modèle entraîné")

Y_pred = model.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
print(f"\n--- Performance du Modèle ---")
print(f"Accuracy (Précision globale) : {accuracy * 100:.2f}%")

cm = confusion_matrix(Y_test, Y_pred)
print("\n--- Matrice de Confusion ---")
print(cm)

Entraînement du modèle...
Modèle entraîné

--- Performance du Modèle ---
Accuracy (Précision globale) : 75.90%

--- Matrice de Confusion ---
[[14138   247]
 [ 4352   347]]


Si on avait un modèle "stupide" qui prédisait "0" (Non-champion) à tous les clients, tout le temps, il aurait une précision de 75.02%.

Notre modèle "intelligent" fait 75.90%.

C'est à peine 0.88% meilleur qu'un modèle qui ne fait rien d'autre que de deviner "non". C'est un échec.

Regardons du côté de la matrice de confusion : 
- Total de vrais "Champions" (1) dans le test : 4352 + 347 = 4699 clients.
- Total de vrais "Non-champions" (0) dans le test : 14138 + 247 = 14385 clients.

Maintenant regardons les prédictions : 
- Vrais Négatifs (VN = 14138) : Il a correctement identifié 14 138 "non-champions". C'est pour ça que l'accuracy est si élevée. Il est génial pour trouver les "0".
- Faux Positifs (FP = 247) : Il s'est trompé 247 fois, en appelant un "0" un "1".
- Vrais Positifs (VP = 347) : Il a correctement trouvé 347 "Champions".
- Faux Négatifs (FN = 4352) : C'est ici l'échec, il y avait 4699 "Champions", et notre modèle en a RATÉ 4352.

On a construit un modèle pour trouver nos meilleurs clients. Il n'en trouve que 7.4% (347 / 4699). 92.6% de nos meilleurs clients sont passés sous le radar.

Ce n'est pas la Régression Logistique qui est mauvaise. C'est notre hypothès : recency et frequency seules ne suffisent pas à prédire la monetary.

In [15]:
df_rfm.to_csv('data/olist_rfm_segments.csv', index=False)